## Importing Libraries

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

**2308. Arrange Table by Gender (Medium)**

**Table: Genders**

| Column Name | Type    |
|-------------|---------|
| user_id     | int     |
| gender      | varchar |

user_id is the primary key (column with unique values) for this table.
gender is ENUM (category) of type 'female', 'male', or 'other'.
Each row in this table contains the ID of a user and their gender.
The table has an equal number of 'female', 'male', and 'other'.
 
**Write a solution to rearrange the Genders table such that the rows alternate between 'female', 'other', and 'male' in order. The table should be rearranged such that the IDs of each gender are sorted in ascending order.**

Return the result table in the mentioned order.

The result format is shown in the following example.

**Example 1:**

**Input:** 

**Genders table:**

| user_id | gender |
|---------|--------|
| 4       | male   |
| 7       | female |
| 2       | other  |
| 5       | male   |
| 3       | female |
| 8       | male   |
| 6       | other  |
| 1       | other  |
| 9       | female |

**Output:** 
| user_id | gender |
|---------|--------|
| 3       | female |
| 1       | other  |
| 4       | male   |
| 7       | female |
| 2       | other  |
| 5       | male   |
| 9       | female |
| 6       | other  |
| 8       | male   |

**Explanation:** 
- Female gender: IDs 3, 7, and 9.
- Other gender: IDs 1, 2, and 6.
- Male gender: IDs 4, 5, and 8.

We arrange the table alternating between 'female', 'other', and 'male'.

Note that the IDs of each gender are sorted in ascending order.

In [0]:
genders_data_2308 = [
    (4, "male"),
    (7, "female"),
    (2, "other"),
    (5, "male"),
    (3, "female"),
    (8, "male"),
    (6, "other"),
    (1, "other"),
    (9, "female"),
]

genders_columns_2308 = ["user_id", "gender"]
genders_df_2308 = spark.createDataFrame(genders_data_2308, genders_columns_2308)
genders_df_2308.show()

+-------+------+
|user_id|gender|
+-------+------+
|      4|  male|
|      7|female|
|      2| other|
|      5|  male|
|      3|female|
|      8|  male|
|      6| other|
|      1| other|
|      9|female|
+-------+------+



In [0]:
window_spec = Window.partitionBy("gender").orderBy("user_id")

In [0]:
df_ranked_2308 = genders_df_2308\
                    .withColumn("rn", row_number().over(window_spec))

In [0]:
df_pivoted_2308 = df_ranked_2308\
                    .groupBy("rn")\
                        .pivot("gender", ["female", "other", "male"])\
                            .agg(first("user_id"))

In [0]:
female_df_2308 = df_pivoted_2308\
                    .select(
                        col("rn"),
                        col("female").alias("user_id"),
                        lit("female").alias("gender")
                        )\
                            .where(col("female").isNotNull())

In [0]:
other_df_2308 = df_pivoted_2308\
                    .select(
                        col("rn"),
                        col("other").alias("user_id"),
                        lit("other").alias("gender")
                        )\
                            .where(col("other").isNotNull())

In [0]:
male_df_2308 = df_pivoted_2308\
                    .select(
                        col("rn"),
                        col("male").alias("user_id"),
                        lit("male").alias("gender")
                        )\
                            .where(col("male").isNotNull())

In [0]:
female_df_2308\
    .unionByName(other_df_2308)\
        .unionByName(male_df_2308).orderBy("rn").drop("rn").show()

+-------+------+
|user_id|gender|
+-------+------+
|      1| other|
|      4|  male|
|      3|female|
|      5|  male|
|      2| other|
|      7|female|
|      8|  male|
|      6| other|
|      9|female|
+-------+------+

